In [ ]:
!pip install --quiet --upgrade git+https://github.com/huggingface/transformers

In [ ]:
!pip install --quiet --upgrade git+https://github.com/huggingface/diffusers

In [ ]:
!pip install --quiet --upgrade git+https://github.com/huggingface/accelerate

In [ ]:
!pip install --quiet --upgrade bitsandbytes

In [ ]:
!pip install --quiet --upgrade sentencepiece

In [ ]:
!pip install --quiet --upgrade protobuf

In [1]:
from diffusers import FluxTransformer2DModel, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True
)

transformer = FluxTransformer2DModel.from_pretrained(
    "black-forest-labs/FLUX.1-dev", 
    subfolder="transformer",
    quantization_config=quantization_config
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
transformer.get_memory_footprint()

5955332224

In [4]:
import utils
utils.print_device_info()
utils.print_model_info(transformer)

driver version : 550.127.08
device count : 2
device 0 : NVIDIA GeForce RTX 4090
device 0 : mem total : 24564 MB
device 0 : mem used  : 7467 MB
device 0 : mem free  : 17096 MB
device 1 : NVIDIA GeForce RTX 4090
device 1 : mem total : 24564 MB
device 1 : mem used  : 1570 MB
device 1 : mem free  : 22993 MB
model : size : 5679 MB
model : is quantized : True
model : quantization method : QuantizationMethod.BITS_AND_BYTES
model : 4-bit quantized : True
model : on GPU : True


In [5]:
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    transformer=transformer,
    device_map="balanced"
)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
from diffusers import FluxPipeline, BitsAndBytesConfig
# import torch

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    quantization_config=quantization_config
    # torch_dtype=torch.bfloat16,
    # use_safetensors=True,
    # device_map="balanced", # sequential not supported
    # max_memory={0:"23.6GB", 1:"23.6GB"} # need to specfy max memory to not over allocate gpu:0
).to("cuda")

print(pipe.hf_device_map)

In [ ]:
from diffusers import FluxPipeline
import torch

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="balanced", # sequential not supported
    max_memory={0:"23.6GB", 1:"23.6GB"} # need to specfy max memory to not over allocate gpu:0
)#.to("cuda")

In [6]:
prompt = "A cat holding a sign that says hello world"

image = pipe(
    prompt,
    width=1024,
    height=768,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=512,
    # generator=torch.Generator("cuda").manual_seed(0)
).images[0]

image

  0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
prompt = "A backrooms liminal space"

image = pipe(
    prompt,
    width=1024,
    height=768,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=512,
    # generator=torch.Generator("cuda").manual_seed(0)
).images[0]

image

In [ ]:
import utils
utils.print_device_info()
# utils.print_model_info(pipe.model) # FluxPipeline does't have a model object

In [ ]:
print(pipe.hf_device_map)

In [ ]:
from diffusers import FluxPipeline
import torch

prompt = "a photo of a dog with cat-like look"

pipeline = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    transformer=None,
    vae=None,
    device_map="balanced",
    max_memory={0: "16GB", 1: "16GB"},
    torch_dtype=torch.bfloat16
)
with torch.no_grad():
    print("Encoding prompts.")
    prompt_embeds, pooled_prompt_embeds, text_ids = pipeline.encode_prompt(
        prompt=prompt, prompt_2=None, max_sequence_length=512
    )

In [ ]:
import gc 

def flush():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.reset_peak_memory_stats()

del pipeline.text_encoder
del pipeline.text_encoder_2
del pipeline.tokenizer
del pipeline.tokenizer_2
del pipeline

flush()

In [ ]:
from diffusers import FluxTransformer2DModel
import torch 

transformer = FluxTransformer2DModel.from_pretrained(
    "black-forest-labs/FLUX.1-dev", 
    subfolder="transformer",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [ ]:
pipeline = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    text_encoder=None,
    text_encoder_2=None,
    tokenizer=None,
    tokenizer_2=None,
    vae=None,
    transformer=transformer,
    torch_dtype=torch.bfloat16
)

print("Running denoising.")
height, width = 768, 1360
latents = pipeline(
    prompt_embeds=prompt_embeds,
    pooled_prompt_embeds=pooled_prompt_embeds,
    num_inference_steps=50,
    guidance_scale=3.5,
    height=height,
    width=width,
    output_type="latent",
).images

In [ ]:
del pipeline.transformer
del pipeline

flush()

In [ ]:
from diffusers import AutoencoderKL
from diffusers.image_processor import VaeImageProcessor
import torch 

vae = AutoencoderKL.from_pretrained("black-forest-labs/FLUX.1-dev", subfolder="vae", torch_dtype=torch.bfloat16).to("cuda")
vae_scale_factor = 2 ** (len(vae.config.block_out_channels))
image_processor = VaeImageProcessor(vae_scale_factor=vae_scale_factor)

with torch.no_grad():
    print("Running decoding.")
    latents = FluxPipeline._unpack_latents(latents, height, width, vae_scale_factor)
    latents = (latents / vae.config.scaling_factor) + vae.config.shift_factor

    image = vae.decode(latents, return_dict=False)[0]
    image = image_processor.postprocess(image, output_type="pil")
    image[0].save("split_transformer.png")